In [146]:
import pandas as pd


In [147]:
#import of review data
cols = ["user id","item id","rating","timestamp"]
#encoding using ISO-8859-1 is used because utf-8 does not support all the characters in movie names
df_data = pd.read_csv("ml-100k/u.data",sep="\t",names=cols,header=None,encoding="ISO-8859-1")

In [ ]:
#verifying the sucessful import of review data
print(df_data.head())

In [148]:
#import of moviedata
cols = ["movie id",
        "movie title",
        "release date",
        "video release date",
        "IMDb URL","unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western"]

df_movie = pd.read_csv("ml-100k/u.item",sep="|",names=cols,header=None,encoding="ISO-8859-1")

In [ ]:
#verifying the sucessful import of movie data
print(df_movie.head())

In [149]:
#import of user data
cols = ["user id","age","gender","occupation","zip code"]
df_user = pd.read_csv("ml-100k/u.user",sep="|",names=cols,header=None,encoding="ISO-8859-1")

In [ ]:
#verifying the sucessful import of user data
print(df_user.head())

In [150]:
#frequency binning the ages into age groups as it will be easier for future analysis
df_user['age_group'] = pd.qcut(df_user['age'],q=10,precision=0)

#the bins are of unequal size due to repeating values in a bin
df_user['age_group'].value_counts()

(6.0, 20.0]     109
(23.0, 26.0]    105
(35.0, 40.0]    100
(31.0, 35.0]     98
(29.0, 31.0]     96
(40.0, 46.0]     94
(46.0, 51.0]     93
(20.0, 23.0]     92
(51.0, 73.0]     85
(26.0, 29.0]     71
Name: age_group, dtype: int64

In [151]:
df_movie.drop(["movie id",
               "movie title",
               "release date",
               "video release date",
               "IMDb URL",
               "unknown"],axis=1).sum(axis = 0, skipna = True)

Action         251
Adventure      135
Animation       42
Children's     122
Comedy         505
Crime          109
Documentary     50
Drama          725
Fantasy         22
Film-Noir       24
Horror          92
Musical         56
Mystery         61
Romance        247
Sci-Fi         101
Thriller       251
War             71
Western         27
dtype: int64

In [152]:
df = pd.merge(pd.merge(df_data,
                  df_user[["user id",
                           "age",
                           "gender",
                           "occupation"]],
                  on='user id',
                  how='left'),
              df_movie.drop(['IMDb URL'],axis=1),
              left_on = 'item id',
              right_on = 'movie id',
              how ='left')
              

In [ ]:
df.head()

In [153]:
def release_year(row):
    return str(row['release date'])[-4:]
    
def release_month(row):
    return str(row['release date'])[3:6]
def release_date(row):
    return str(row['release date'])[:2]

df['release_year'] =  df.apply(lambda row: release_year(row), axis=1)
df['release_month'] =  df.apply(lambda row: release_month(row), axis=1)
df['release_day'] =  df.apply(lambda row: release_date(row), axis=1)

df = df.drop(['release date'])

KeyError: "['release date'] not found in axis"

In [154]:
df_job_genre = df[['occupation',
                   'rating',
                   "Action",
                   "Adventure",
                   "Animation",
                   "Children's",
                   "Comedy",
                   "Crime",
                   "Documentary",
                   "Drama",
                   "Fantasy",
                   "Film-Noir",
                   "Horror",
                   "Musical",
                   "Mystery",
                   "Romance",
                   "Sci-Fi",
                   "Thriller",
                   "War",
                   "Western"]]


In [155]:
def select_genre(row):
    for key,value in row.items():
        if value==1:
            return key

In [156]:
df_job_genre['genre']= df_job_genre.apply(lambda row: select_genre(row.iloc[2:]),axis=1)
df_job_genre.drop(["Action",
                   "Adventure",
                   "Animation",
                   "Children's",
                   "Comedy",
                   "Crime",
                   "Documentary",
                   "Drama",
                   "Fantasy",
                   "Film-Noir",
                   "Horror",
                   "Musical",
                   "Mystery",
                   "Romance",
                   "Sci-Fi",
                   "Thriller",
                   "War",
                   "Western"],
                 inplace=True,
                 axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [199]:
df_job_genre_grouped = df_job_genre.groupby(['occupation','genre']).mean()

In [202]:
df_favorite_genre = df_job_genre_grouped.reset_index(level='occupation').groupby('occupation')['rating'].idxmax().reset_index(name='favorite_genre')

In [203]:
df_favorite_genre.head()

,occupation,favorite_genre
0,administrator,War
1,artist,Film-Noir
2,doctor,Film-Noir
3,educator,Film-Noir
4,engineer,Film-Noir
